In [1]:
# interactive_reco_amazon2018_simple_fix.py
# Digital_Music -> Movies_and_TV recommender (hidden meta + params)
# Fix: avoid using `state.value` inside callbacks; use `st` (the State payload) correctly.

import os
import json
import gzip
from pathlib import Path
from typing import Dict, List, Optional, Any, Tuple

import gradio as gr
import numpy as np
import pandas as pd

# --------- Paths (defaults).  can still override via environment variables ---------
BASE = os.environ.get("CDR_BASE", "/data/nas07/PersonalData/farchan/CDR/cdr_project")
DEFAULT_RUN_DIR = os.environ.get("CDR_RUN_DIR", f"{BASE}/checkpoint/MF/exp-mf64-2")
DEFAULT_ART_DIR = os.environ.get("CDR_ART_DIR", f"{BASE}/artifacts/movies_from_music-2")

# From r dataset:
DEFAULT_DM_META = os.environ.get(
    "CDR_DM_META",
    "data/nas07/PersonalData/farchan/CDR/cdr_project/data/amazon2018/metadata/meta_Digital_Music.json.gz"
)
DEFAULT_MV_META = os.environ.get(
    "CDR_MV_META",
    "data/nas07/PersonalData/farchan/CDR/cdr_project/data/amazon2018/metadata/meta_Movies_and_TV.json.gz"
)

# --------- Fixed session parameters (hidden from UI) ---------
UI_DEFAULTS = {
    "k": 10,
    "topn": 400,
    "mmr": 0.0,
    "w_user": 1.0,
    "w_like": 1.2,
    "w_dislike": 0.6,
    "lr": 0.05,
    "reg": 1e-4,
    "steps_fb": 5,
}

def _resolve_path(p: str) -> str:
    if Path(p).exists():
        return p
    if not p.startswith("/"):
        p2 = "/" + p
        if Path(p2).exists():
            return p2
    return p

# ---------------- Core session re-ranker ----------------
class SessionSearch:
    def __init__(self, U: np.ndarray, V: np.ndarray, user_pos_all: Dict[int, set], target_pool: np.ndarray):
        self.U = U.astype(np.float32, copy=False)
        self.V = V.astype(np.float32, copy=False)
        self.user_pos_all = user_pos_all
        self.pool = np.array(target_pool, dtype=np.int64)

        self.uid: Optional[int] = None
        self.delta = np.zeros(U.shape[1], dtype=np.float32)
        self.liked: List[int] = []
        self.disliked: List[int] = []

        # runtime-tunable (but fixed via UI_DEFAULTS here)
        self.w_user = UI_DEFAULTS["w_user"]
        self.w_like = UI_DEFAULTS["w_like"]
        self.w_dislike = UI_DEFAULTS["w_dislike"]
        self.lambda_mmr = UI_DEFAULTS["mmr"]
        self.lr = UI_DEFAULTS["lr"]
        self.reg = UI_DEFAULTS["reg"]
        self.steps_per_feedback = UI_DEFAULTS["steps_fb"]

        self.u_base = self.U.mean(axis=0).astype(np.float32)

    def start_cold(self):
        self.uid = -1
        self.delta[:] = 0.0
        self.liked.clear()
        self.disliked.clear()

    def _user_vec(self) -> np.ndarray:
        return (self.u_base + self.delta).astype(np.float32, copy=False)

    def _session_vec(self) -> np.ndarray:
        s = self.w_user * self._user_vec()
        if self.liked:
            s = s + self.w_like * self.V[np.array(self.liked, dtype=np.int64)].mean(axis=0)
        if self.disliked:
            s = s - self.w_dislike * self.V[np.array(self.disliked, dtype=np.int64)].mean(axis=0)
        return s.astype(np.float32, copy=False)

    @staticmethod
    def _cosine(a: np.ndarray, b: np.ndarray, eps: float = 1e-8) -> float:
        return float(a.dot(b) / ((np.linalg.norm(a) + eps) * (np.linalg.norm(b) + eps)))

    def _mmr_select(self, cand_ids: np.ndarray, scores: np.ndarray, k: int) -> List[int]:
        lam = float(self.lambda_mmr)
        if lam <= 0.0 or k <= 1:
            order = np.argsort(-scores)[:k]
            return cand_ids[order].tolist()
        selected: List[int] = []
        selected_vecs: List[np.ndarray] = []
        remain = list(range(len(cand_ids)))
        for _ in range(min(k, len(remain))):
            best_idx, best_val = None, -1e18
            for ridx in remain:
                i = cand_ids[ridx]
                vi = self.V[i]
                rel = float(scores[ridx])
                div = 0.0
                if selected_vecs:
                    div = max(self._cosine(vi, vj) for vj in selected_vecs)
                val = lam * rel - (1.0 - lam) * div
                if val > best_val:
                    best_val, best_idx = val, ridx
            chosen = cand_ids[best_idx]
            selected.append(int(chosen))
            selected_vecs.append(self.V[chosen])
            remain.remove(best_idx)
        return selected

    def feedback_like(self, iid: int):
        iid = int(iid)
        if iid in self.disliked:
            self.disliked.remove(iid)
        if iid not in self.liked:
            self.liked.append(iid)
        self._personalize()

    def feedback_dislike(self, iid: int):
        iid = int(iid)
        if iid in self.liked:
            self.liked.remove(iid)
        if iid not in self.disliked:
            self.disliked.append(iid)
        self._personalize()

    def feedback_rating(self, iid: int, r: int):
        iid, r = int(iid), int(r)
        if r >= 4:
            self.feedback_like(iid)
        elif r <= 2:
            self.feedback_dislike(iid)
        else:
            if iid in self.liked:
                self.liked.remove(iid)
            if iid in self.disliked:
                self.disliked.remove(iid)
            self._personalize(steps=max(1, self.steps_per_feedback // 2))

    def _personalize(self, steps: Optional[int] = None):
        if not self.liked or not self.disliked:
            return
        steps = int(self.steps_per_feedback if steps is None else steps)
        for _ in range(steps):
            pos = self.V[np.random.choice(self.liked)]
            neg = self.V[np.random.choice(self.disliked)]
            u = self._user_vec()
            x = float(u.dot(pos - neg))
            sig = 1.0 / (1.0 + np.exp(-x))
            grad = (sig - 1.0) * (pos - neg) + self.reg * self.delta
            self.delta = (self.delta - self.lr * grad).astype(np.float32, copy=False)

    def recommend(self, k: int = None, topn: int = None) -> List[Tuple[int, float]]:
        if k is None: k = UI_DEFAULTS["k"]
        if topn is None: topn = UI_DEFAULTS["topn"]
        pool = self.pool
        if len(pool) == 0:
            return []
        base = self.V[pool] @ self._user_vec()
        if len(pool) > topn:
            idx = np.argpartition(-base, topn)[:topn]
            cand, _ = pool[idx], base[idx]
        else:
            cand = pool
        svec = self._session_vec()
        final = self.V[cand] @ svec
        slate = self._mmr_select(cand, final, k)
        score_map = {int(i): float(final[j]) for j, i in enumerate(cand)}
        return [(int(i), score_map[int(i)]) for i in slate]

# ---------------- Helpers for labels ----------------
GENERIC_TOKENS = {
    "movies & tv", "movies and tv", "movie & tv", "general", "top sellers",
    "science fiction & fantasy", "science fiction", "fantasy", "action",
    "comedy", "drama", "horror", "kids & family", "children & family",
    "studio specials", "spanish language", "television", "tv",
    "blu-ray", "dvd", "4k", "uhd", "collector's edition", "box set"
}
GENRE_WHITELIST = {
    "action", "adventure", "comedy", "romance", "drama", "horror", "thriller",
    "mystery", "animation", "anime", "documentary", "biography", "music",
    "family", "kids & family", "science fiction & fantasy", "science fiction",
    "fantasy", "sci-fi", "sci-fi action", "crime", "war", "western", "sports"
}

def _tokenize_cat(cat: Any) -> List[str]:
    if isinstance(cat, list):
        tokens = [str(x) for x in cat]
    else:
        s = str(cat)
        if s.startswith("[") and s.endswith("]"):
            inner = s[1:-1].strip()
            parts = [p.strip().strip("'\"") for p in inner.split(",")]
            tokens = [p for p in parts if p]
        elif ">" in s:
            tokens = [t.strip() for t in s.split(">")]
        else:
            tokens = [s]
    seen = set(); out = []
    for t in tokens:
        tl = t.lower()
        if tl not in seen:
            seen.add(tl); out.append(t)
    return out

def _looks_like_title(t: str) -> bool:
    return (" " in t or any(ch.isdigit() for ch in t)) and len(t) >= 3 and t.lower() not in GENERIC_TOKENS

def _extract_genre(tokens: List[str]) -> Optional[str]:
    for t in tokens:
        tl = t.lower()
        if tl in GENRE_WHITELIST:
            if tl == "science fiction & fantasy" or tl == "science fiction":
                return "Sci-Fi & Fantasy"
            if tl == "kids & family":
                return "Family"
            if tl == "sci-fi":
                return "Sci-Fi"
            if tl == "sci-fi action":
                return "Sci-Fi Action"
            return t
    return None

# ---------------- Meta readers ----------------
def _read_meta_json_gz(path: str, needed_asins: set, want_categories: bool = False) -> Dict[str, dict]:
    out = {}
    p = Path(path)
    if not p.exists():
        return out
    with gzip.open(p, "rt", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                o = json.loads(line)
            except Exception:
                continue
            asin = o.get("asin")
            if not asin or asin not in needed_asins:
                continue
            rec = {}
            if "title" in o and isinstance(o["title"], str):
                rec["title"] = o["title"].strip()
            if "brand" in o and isinstance(o["brand"], str):
                rec["brand"] = o["brand"].strip()
            if want_categories:
                cats = None
                if "categories" in o:
                    if isinstance(o["categories"], list) and o["categories"] and isinstance(o["categories"][0], list):
                        cats = [c for sub in o["categories"] for c in sub]
                    elif isinstance(o["categories"], list):
                        cats = [str(x) for x in o["categories"]]
                if not cats and "category" in o:
                    cats = [str(o["category"])]
                if cats:
                    rec["categories"] = cats
            out[asin] = rec
    return out

# ---------------- Label builders ----------------
def label_music(iid: int, aux: dict) -> str:
    mp = aux.get("music_label", {})
    if iid in mp and mp[iid]:
        return mp[iid]
    dec = aux["decoder"].get(iid, str(iid))
    return dec.split("::", 1)[-1]

def label_movie(iid: int, aux: dict) -> str:
    titles = aux.get("movie_titles", {})
    if iid in titles and titles[iid]:
        title = titles[iid]
        cat = aux.get("movie_cats", {}).get(iid)
        if cat:
            tokens = _tokenize_cat(cat)
            g = _extract_genre(tokens)
            if g:
                return f"{title} • {g}"
        return title
    cat = aux.get("movie_cats", {}).get(iid)
    if cat:
        tokens = _tokenize_cat(cat)
        for t in tokens:
            if _looks_like_title(t):
                g = _extract_genre(tokens)
                return f"{t} • {g}" if g else t
    dec = aux["decoder"].get(iid, str(iid))
    return dec.split("::", 1)[-1]

# ---------------- Load artifacts + meta ----------------
def load_artifacts(run_dir: Path, artifacts_dir: Path):
    run_dir = Path(run_dir); artifacts_dir = Path(artifacts_dir)
    U = np.load(run_dir / "user_embeddings.npy")
    V = np.load(run_dir / "item_embeddings.npy")

    data_dir = artifacts_dir / "data"
    train = pd.read_parquet(data_dir / "train.parquet")
    val = pd.read_parquet(data_dir / "val.parquet")
    test = pd.read_parquet(data_dir / "test.parquet")

    user_pos_all = {
        int(u): set(map(int, g["iid"]))
        for u, g in pd.concat([train[["uid","iid"]], val[["uid","iid"]], test[["uid","iid"]]], ignore_index=True).groupby("uid")
    }

    with open(artifacts_dir / "maps" / "item_id_map.json", "r") as f:
        iid_map = json.load(f)
    decoder = {int(v): k for k, v in iid_map.items()}

    all_iids = pd.concat([train["iid"], val["iid"], test["iid"]]).unique().tolist()
    valtest_iids = pd.concat([val["iid"], test["iid"]]).unique().tolist()

    def domain_of(i):
        key = decoder.get(int(i), str(i))
        return key.split("::", 1)[0] if "::" in key else "UNKNOWN"

    domains = sorted({domain_of(i) for i in all_iids})
    domain_to_all = {d: [] for d in domains}
    domain_to_valtest = {d: [] for d in domains}
    for i in all_iids:
        domain_to_all[domain_of(i)].append(int(i))
    for i in valtest_iids:
        domain_to_valtest[domain_of(i)].append(int(i))

    default_target_domain = "Movies_and_TV" if "Movies_and_TV" in domains else (domains[0] if domains else "UNKNOWN")
    target_pool = np.array(sorted(domain_to_valtest.get(default_target_domain, [])), dtype=np.int64)

    # Build asin sets per domain
    dom_asins: Dict[str, set] = {d: set() for d in domains}
    for iid in all_iids:
        key = decoder.get(int(iid), str(iid))
        if "::" in key:
            dom, asin = key.split("::", 1)
            dom_asins.setdefault(dom, set()).add(asin)

    # Resolve meta paths (hidden; no UI)
    dm_meta_path = _resolve_path(DEFAULT_DM_META)
    mv_meta_path = _resolve_path(DEFAULT_MV_META)

    # Enrich MUSIC (Digital_Music)
    music_label: Dict[int, str] = {}
    dm_dom_keys = [d for d in domains if d.lower().replace(" ", "_") == "digital_music"]
    dm_dom = dm_dom_keys[0] if dm_dom_keys else None
    if dm_dom and Path(dm_meta_path).exists():
        needed = dom_asins.get(dm_dom, set())
        meta = _read_meta_json_gz(dm_meta_path, needed_asins=needed, want_categories=False)
        for iid in domain_to_all.get(dm_dom, []):
            key = decoder.get(iid, str(iid))
            asin = key.split("::", 1)[-1]
            m = meta.get(asin, {})
            title = m.get("title")
            brand = m.get("brand")
            if brand and title:
                music_label[iid] = f"{brand} – {title}"
            elif title:
                music_label[iid] = title
            elif brand:
                music_label[iid] = brand

    # Enrich MOVIES
    movie_titles: Dict[int, str] = {}
    movie_cats: Dict[int, Any] = {}
    for df in (val, test):
        for col in ("title","item_title","product_title","name"):
            if col in df.columns:
                for row in df[["iid", col]].itertuples(index=False):
                    t = str(getattr(row, col)).strip()
                    if t and t.lower() != "nan":
                        movie_titles[int(row.iid)] = t
                break
        if "item_cat" in df.columns:
            for row in df[["iid","item_cat"]].itertuples(index=False):
                movie_cats[int(row.iid)] = row.item_cat

    mv_dom_keys = [d for d in domains if d.lower().replace(" ", "_") == "movies_and_tv"]
    mv_dom = mv_dom_keys[0] if mv_dom_keys else None
    if mv_dom and Path(mv_meta_path).exists():
        needed = dom_asins.get(mv_dom, set())
        meta = _read_meta_json_gz(mv_meta_path, needed_asins=needed, want_categories=True)
        for iid in domain_to_all.get(mv_dom, []):
            key = decoder.get(iid, str(iid))
            asin = key.split("::", 1)[-1]
            m = meta.get(asin, {})
            if "title" in m and iid not in movie_titles:
                movie_titles[iid] = m["title"]
            if "categories" in m and iid not in movie_cats:
                movie_cats[iid] = m["categories"]

    aux = {
        "domains": domains,
        "domain_to_all": {k: np.array(sorted(v), dtype=np.int64) for k, v in domain_to_all.items()},
        "domain_to_valtest": {k: np.array(sorted(v), dtype=np.int64) for k, v in domain_to_valtest.items()},
        "default_target_domain": default_target_domain,
        "decoder": decoder,
        "music_label": music_label,
        "movie_titles": movie_titles,
        "movie_cats": movie_cats,
    }
    return U, V, user_pos_all, target_pool, aux

# ---------------- Gradio App (minimal UI) ----------------
with gr.Blocks(title="Digital_Music → Movies_and_TV") as demo:
    gr.Markdown("Select **Digital_Music** seeds → get **Movies_and_TV** recommendations. "
                "Meta paths and tuning params are preconfigured (hidden).")

    state = gr.State(value={"sess": None, "pool": None, **{}})

    with gr.Row():
        run_dir = gr.Textbox(value=DEFAULT_RUN_DIR, label="Run directory")
        art_dir = gr.Textbox(value=DEFAULT_ART_DIR, label="Artifacts directory")
    load_btn = gr.Button("Load", variant="primary")
    load_info = gr.Markdown()

    with gr.Row():
        src_domain = gr.Dropdown(choices=[], label="Source domain", value=None)
        tgt_domain = gr.Dropdown(choices=[], label="Target domain", value=None)
        use_valtest = gr.Checkbox(value=True, label="Use val/test target pool")

    with gr.Row():
        filter_txt = gr.Textbox(value="", label="Filter Digital_Music by text")
        max_show = gr.Slider(50, 1000, value=400, step=50, label="Max items to show")
        refresh_src = gr.Button("Refresh list")
    src_items = gr.Dropdown(choices=[], multiselect=True, label="Pick 1+ Digital_Music items")

    rec_btn = gr.Button("Recommend", variant="secondary")
    rec_table = gr.Dataframe(headers=["iid", "score", "label"], row_count=(0, "dynamic"), col_count=3, wrap=True, interactive=False)
    select_iid = gr.Dropdown(choices=[], label="Pick a movie to feedback")

    with gr.Row():
        like_btn = gr.Button("👍 Like")
        dislike_btn = gr.Button("👎 Dislike")
        rating = gr.Slider(1, 5, value=3, step=1, label="Rate 1..5")
        rate_btn = gr.Button("Rate")
        reset_btn = gr.Button("Reset session")
    fb_info = gr.Markdown("")

    def on_load(run_dir_str, art_dir_str):
        try:
            U, V, user_pos_all, target_pool, aux = load_artifacts(run_dir_str, art_dir_str)
            sess = SessionSearch(U, V, user_pos_all, target_pool)
            sess.start_cold()
            domains = aux["domains"]
            src_default = "Digital_Music" if "Digital_Music" in domains else (domains[0] if domains else None)
            tgt_default = "Movies_and_TV" if "Movies_and_TV" in domains else aux["default_target_domain"]
            st = {"sess": sess, "pool": target_pool, "aux": aux}
            msg = (f"✅ Loaded. Users={U.shape[0]:,} Items={V.shape[0]:,} Domains={', '.join(domains)} "
                   f"| Music meta labels: {len(aux.get('music_label', {}))} "
                   f"| Movie titles: {len(aux.get('movie_titles', {}))}")
            return msg, gr.update(choices=domains, value=src_default), gr.update(choices=domains, value=tgt_default), st
        except Exception as e:
            return f"❌ Failed to load: {e}", gr.update(), gr.update(), {}

    load_btn.click(on_load, inputs=[run_dir, art_dir], outputs=[load_info, src_domain, tgt_domain, state])

    def refresh_source_list(src_dom, filt, max_items, st):
        aux = st.get("aux", {})
        if st.get("sess") is None or not src_dom:
            return gr.update(choices=[], value=[])
        dom_ids = aux["domain_to_all"].get(src_dom, [])
        labels = [label_music(int(i), aux) for i in dom_ids]
        if filt:
            f = str(filt).lower()
            pairs = [(lab, int(i)) for lab, i in zip(labels, dom_ids) if f in lab.lower()]
        else:
            pairs = [(lab, int(i)) for lab, i in zip(labels, dom_ids)]
        pairs = pairs[: int(max_items)]
        return gr.update(choices=pairs, value=[])

    refresh_src.click(refresh_source_list, inputs=[src_domain, filter_txt, max_show, state], outputs=[src_items])

    def _set_target_pool(sess: SessionSearch, st, tgt_dom: str, use_valtest_only: bool):
        aux = st["aux"]
        pool = aux["domain_to_valtest"].get(tgt_dom, []) if use_valtest_only else aux["domain_to_all"].get(tgt_dom, [])
        sess.pool = np.array(pool, dtype=np.int64)

    def on_seed_recommend(seed_iids, src_dom, tgt_dom, use_valtest_only, st):
        sess: SessionSearch = st["sess"]; aux = st["aux"]
        if sess is None:
            return "Load artifacts first.", pd.DataFrame(), gr.update(choices=[])
        if not tgt_dom:
            return "Pick a target domain.", pd.DataFrame(), gr.update(choices=[])
        sess.start_cold()
        _set_target_pool(sess, st, str(tgt_dom), bool(use_valtest_only))
        # apply defaults (hidden)
        sess.w_user = UI_DEFAULTS["w_user"]; sess.w_like = UI_DEFAULTS["w_like"]; sess.w_dislike = UI_DEFAULTS["w_dislike"]
        sess.lambda_mmr = UI_DEFAULTS["mmr"]; sess.lr = UI_DEFAULTS["lr"]; sess.reg = UI_DEFAULTS["reg"]; sess.steps_per_feedback = UI_DEFAULTS["steps_fb"]
        seeds = seed_iids or []
        if isinstance(seeds, int):
            seeds = [seeds]
        for iid in seeds:
            sess.feedback_like(int(iid))
        recs = sess.recommend(k=UI_DEFAULTS["k"], topn=UI_DEFAULTS["topn"])
        rows, choices = [], []
        for iid_val, score in recs:
            lab = label_movie(int(iid_val), aux)
            rows.append([iid_val, round(score, 4), lab])
            choices.append((lab, iid_val))
        df = pd.DataFrame(rows, columns=["iid", "score", "label"])
        msg = f"🎯 Seeds: {len(seeds)} | Target={tgt_dom}"
        return msg, df, gr.update(choices=choices, value=(choices[0][1] if choices else None))

    rec_btn.click(on_seed_recommend, inputs=[src_items, src_domain, tgt_domain, use_valtest, state], outputs=[fb_info, rec_table, select_iid])

    def _refresh_after(action: str, iid_val, st):
        sess: SessionSearch = st["sess"]; aux = st["aux"]
        if sess is None or iid_val is None:
            return "Select an item first.", pd.DataFrame(), gr.update(choices=[])
        # re-apply defaults (hidden)
        sess.w_user = UI_DEFAULTS["w_user"]; sess.w_like = UI_DEFAULTS["w_like"]; sess.w_dislike = UI_DEFAULTS["w_dislike"]
        sess.lambda_mmr = UI_DEFAULTS["mmr"]; sess.lr = UI_DEFAULTS["lr"]; sess.reg = UI_DEFAULTS["reg"]; sess.steps_per_feedback = UI_DEFAULTS["steps_fb"]
        if action == "like":
            sess.feedback_like(int(iid_val))
            msg = "👍 Liked."
        elif action == "dislike":
            sess.feedback_dislike(int(iid_val))
            msg = "👎 Disliked."
        else:
            msg = "Updated."
        recs = sess.recommend(k=UI_DEFAULTS["k"], topn=UI_DEFAULTS["topn"])
        rows, choices = [], []
        for iid2, score in recs:
            lab = label_movie(int(iid2), aux)
            rows.append([iid2, round(score, 4), lab])
            choices.append((lab, iid2))
        df = pd.DataFrame(rows, columns=["iid", "score", "label"])
        return msg, df, gr.update(choices=choices, value=(choices[0][1] if choices else None))

    def on_like(iid_val, st):
        return _refresh_after("like", iid_val, st)

    def on_dislike(iid_val, st):
        return _refresh_after("dislike", iid_val, st)

    def on_rate_and_refresh(iid_val, rating_val, st):
        sess: SessionSearch = st["sess"]; aux = st["aux"]
        if sess is None or iid_val is None:
            return "Select an item first.", pd.DataFrame(), gr.update(choices=[])
        sess.w_user = UI_DEFAULTS["w_user"]; sess.w_like = UI_DEFAULTS["w_like"]; sess.w_dislike = UI_DEFAULTS["w_dislike"]
        sess.lambda_mmr = UI_DEFAULTS["mmr"]; sess.lr = UI_DEFAULTS["lr"]; sess.reg = UI_DEFAULTS["reg"]; sess.steps_per_feedback = UI_DEFAULTS["steps_fb"]
        sess.feedback_rating(int(iid_val), int(rating_val))
        recs = sess.recommend(k=UI_DEFAULTS["k"], topn=UI_DEFAULTS["topn"])
        rows, choices = [], []
        for iid2, score in recs:
            lab = label_movie(int(iid2), aux)
            rows.append([iid2, round(score, 4), lab])
            choices.append((lab, iid2))
        df = pd.DataFrame(rows, columns=["iid", "score", "label"])
        return f"⭐ Rated {int(rating_val)}.", df, gr.update(choices=choices, value=(choices[0][1] if choices else None))

    like_btn.click(on_like, inputs=[select_iid, state], outputs=[fb_info, rec_table, select_iid])
    dislike_btn.click(on_dislike, inputs=[select_iid, state], outputs=[fb_info, rec_table, select_iid])
    rate_btn.click(on_rate_and_refresh, inputs=[select_iid, rating, state], outputs=[fb_info, rec_table, select_iid])

    def on_reset(st):
        if st.get("sess") is None:
            return "Load artifacts first."
        st["sess"].start_cold()
        return "🔁 Session reset."
    reset_btn.click(on_reset, inputs=[state], outputs=[fb_info])


/opt/miniconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
demo.launch(share=True, inline=False)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://8d0996d0bad8f9dddb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
